In [1]:
import pandas as pd
import os
from datetime import date
from datetime import datetime 
import numpy as np

"""
El proposito de este script es predecir los shares de las farmacias donde no se realiza venta

la funcion final objetivo es:

predict_shares(meses, mercado)

lo que hace esta funcion es predecir los shares de los productos de Genomma Lab. 
del mercado tomando los ultimos meses
"""

direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Documentos/ECD/Seminario/TP/'

labo = 'Genomma Lab.'

mercado = 'TAFIROL'

#Leo el archivo transformado
base = pd.read_csv(direc + 'base.csv')

#Leo el archivo del ruteo con info de las farmacias
ruteo = pd.read_csv(direc + 'CPAs Visitados.csv', usecols = [0,3,4,5,6], encoding='latin-1', sep = ';')

# 1. Preparacion de los datos

## 1.1 Seleccion de las filas a utilizar

In [2]:
#No uso todos los meses ya que se tienen 25 meses

#cantidad de meses que voy a utilizar
meses = 6

#tomo los meses, los ordeno y agarro los ultimos que quiero
t = pd.unique(base['Año_Mes'])
t = sorted(t, key=lambda x: datetime.strptime(x, "%Y-%m").strftime("%Y-%m"))
t = t[-meses:]
base = base[base['Año_Mes'].isin(t)]

#sumo, me saco de encima la variable Año_Mes y calculo el promedio facturado
base = base.groupby(['Mercado','cpa', 'Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento'],
                    as_index = False)['Monto'].sum()

## 1.2 Agrego ceros a las farmacias donde no vendo mis productos

In [3]:
#el dataset para realizar el modelo se llamara data
#filtro los datos del mercado a utilizar y remuevo la columna 'Mercado'
data = base[base['Mercado'].isin([mercado])]
data.drop(['Mercado'], axis=1, inplace = True)

C:\Users\rvozzi\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [4]:
# aca se puede ver la distribucion de los productos del mercado de data
#TODO: hacer grafica de barras con los 10 productos mas vendidos
tabla = data.groupby(['pres_desc',],as_index = False)['cpa'].nunique()
tabla['distribucion'] = tabla['cpa']/len(pd.unique(data['cpa']))
tabla

,pres_desc,cpa,distribucion
0,ACTRON 600 RAPIDA ACCION 10 caps.blandas.,11682,0.752900
1,ACTRON 600 RAPIDA ACCION 20 caps.blandas.,13883,0.894754
2,ACTRON PLUS caps.x 8,11405,0.735048
3,ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,8947,0.576631
4,ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,11724,0.755607
...,...,...,...
103,TAFIROL comp.blisterx40 x10 comp,207,0.013341
104,TAFIROL comp.x 10,9267,0.597254
105,TAFIROL comp.x 100,4343,0.279905
106,TAFIROL comp.x 30,8713,0.561549


In [5]:
#LLeno la base del mercado con ceros de las farmacias que le faltan a nuestros productos

productos_col = ['Marca', 'Submarca', 'pres_desc', 'lab', 'Segmento']
productos = data[data['lab'].isin([labo])][productos_col].drop_duplicates()

for x in productos['pres_desc']:

    a_list = pd.unique(data[data['pres_desc'].isin([x])]['cpa'])
    b_list = pd.unique(data['cpa'])
    agregar = list(set(b_list)-set(a_list))

    df = pd.DataFrame()
    df['cpa'] = np.array(agregar)
    df['Monto'] = 0

    for col in productos.columns:
        df[col] = productos[productos['pres_desc'].isin([x])][col].values[0]

    data = pd.concat([data, df], ignore_index=True)

In [6]:
data

,cpa,Marca,Submarca,pres_desc,lab,Segmento,Monto
0,A4126AHA,ACTRON,ACTRON 600 RAPIDA ACCION,ACTRON 600 RAPIDA ACCION 10 caps.blandas.,Bayer Consumer,PLUS,1692
1,A4126AHA,ACTRON,ACTRON 600 RAPIDA ACCION,ACTRON 600 RAPIDA ACCION 20 caps.blandas.,Bayer Consumer,PLUS,4095
2,A4126AHA,ACTRON,ACTRON PLUS,ACTRON PLUS caps.x 8,Bayer Consumer,MIGRA,276
3,A4126AHA,ACTRON,ACTRON RAPIDA ACCION,ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,Bayer Consumer,PLUS,236
4,A4126AHA,ACTRON,ACTRON RAPIDA ACCION,ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,Bayer Consumer,PLUS,450
...,...,...,...,...,...,...,...
889287,G4200DKC,TAFIROL,TAFIROL,TAFIROL comp.blisterx40 x10 comp,Genomma Lab.,500 MG,0
889288,T4000HPO,TAFIROL,TAFIROL,TAFIROL comp.blisterx40 x10 comp,Genomma Lab.,500 MG,0
889289,B1685ACP,TAFIROL,TAFIROL,TAFIROL comp.blisterx40 x10 comp,Genomma Lab.,500 MG,0
889290,S2919BVP,TAFIROL,TAFIROL,TAFIROL comp.blisterx40 x10 comp,Genomma Lab.,500 MG,0


## 1.3 Hago el join con el ruteo para ver la data de las farmacias

In [7]:
data = pd.merge(data,
                ruteo,
                left_on = 'cpa',
                right_on = 'CPA',
                how = 'inner')
data.drop(['CPA'], axis=1, inplace = True)

# 2. Seleccion de Variables

## 2.1 One Hot Encoding de la Descripcion, Marca, Segmento, Lab., Provincia, Localidad, Ruta y Visita

In [8]:
onehot_vars = pd.concat([pd.get_dummies(data.pres_desc, prefix='es_'), 
                         pd.get_dummies(data.Marca, prefix='es_'), 
                         pd.get_dummies(data.Segmento, prefix='es_'),
                         pd.get_dummies(data.lab, prefix='es_lab_'),
                         pd.get_dummies(data.PROVINCIA, prefix='es_prov_'), 
                         pd.get_dummies(data.LOCALIDAD, prefix='es_loc_'), 
                         pd.get_dummies(data.RUTA, prefix='es_ruta_'),
                         pd.get_dummies(data.VISITA, prefix='es_visita_')], 
                         axis=1)
onehot_vars

,es__ACTRON 600 RAPIDA ACCION 10 caps.blandas.,es__ACTRON 600 RAPIDA ACCION 20 caps.blandas.,es__ACTRON PLUS caps.x 8,es__ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,es__ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,es__ALIDASE 500mg comp.x 10,es__ANAFLEX PLUS RA caps bl x 8,es__ANAFLEX PLUS RA caps.bl.x 16,es__ANAFLEX caps.blandas x 16,es__ANAFLEX caps.blandas x 8,...,es_ruta__LITORAL,es_ruta__MENDOZA,es_ruta__NEA,es_ruta__NOA NORTE,es_ruta__NOA SUR,es_ruta__PATAGONIA,es_ruta__ROSARIO,es_visita__NO ATENDIDAS,es_visita__NO VISITADA,es_visita__VISITADA
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889288,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
889290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## 2.2 Shares

### 2.2.1 Shares a nivel Pais del Producto

In [9]:
def shares_pais_hier(hier):
    shares_pais_hier = data.groupby([hier],as_index = False)['Monto'].sum()
    col_name = 'share_pais_' + hier
    shares_pais_hier[col_name] = shares_pais_hier['Monto'] / shares_pais_hier['Monto'].sum()
    shares_pais_hier.drop(['Monto'], axis = 1, inplace = True)
    return shares_pais_hier

### 2.2.2 Shares a nivel Localidad y Ruta del Producto

In [10]:
def calculate_shares_geo_hier(hier, geo):
    fact_geo = data.groupby([geo],as_index = False)['Monto'].sum()
    
    shares_geo_hier = data.groupby([geo, hier],as_index = False)['Monto'].sum()
    shares_geo_hier = pd.merge(shares_geo_hier,
                               fact_geo,
                               left_on = geo,
                               right_on = geo,
                               how = 'inner')
    col_name = 'share_' + geo + '_' + hier
    shares_geo_hier[col_name] = shares_geo_hier['Monto_x'] / shares_geo_hier['Monto_y']
    shares_geo_hier.drop(['Monto_x', 'Monto_y'], axis=1, inplace = True)
    return shares_geo_hier

## 2.3 La Farmacia Vende otras marcas Genomma

In [11]:
pivot_gen = pd.pivot_table(base[base['lab'].isin([labo])], 
                           values = 'lab', 
                           index = 'cpa', 
                           columns = 'Marca', 
                           aggfunc=pd.Series.nunique, 
                           fill_value=0)
pivot_gen['Total_gen'] = pivot_gen[pivot_gen.columns].sum(axis=1)
pivot_gen = pivot_gen.add_prefix('vende_')
pivot_gen

Marca,vende_BENGUE,vende_LAFEDRYL,vende_LOMECAN,vende_MULTI-O,vende_NEXT,vende_NEXT AB,vende_NIKZON,vende_PIECIDEX,vende_QG5,vende_SEROCUTINA,vende_SLEEPY,vende_TAFIROL,vende_TAFIROL PEDIATRICO,vende_TUKOL,vende_UNESIA,vende_X-RAY,vende_Total_gen
cpa,,,,,,,,,,,,,,,,,
A4126AHA,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,5
A4126AHE,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,0,9
A4126AKR,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,0,12
A4190AAA,1,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,7
A4190ABG,1,0,1,1,1,1,0,1,1,1,0,1,1,1,1,0,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z9405QGQ,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
Z9407CJF,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,4
Z9407DAD,1,0,1,1,0,0,0,1,1,1,0,1,0,1,1,0,9


## 2.4 Cuantos productos y Marcas todos los Mercados vende la farmacia

In [12]:
cant_total = base[base['cpa'].isin(pd.unique(data['cpa']))].groupby(['cpa'],as_index = False)['pres_desc', 'Marca'].nunique()
cant_total.columns = ['cpa', 'desc_del_tot', 'marcas_del_tot']
cant_total

C:\Users\rvozzi\AppData\Local\Temp/ipykernel_12160/3180449058.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cant_total = base[base['cpa'].isin(pd.unique(data['cpa']))].groupby(['cpa'],as_index = False)['pres_desc', 'Marca'].nunique()


,cpa,desc_del_tot,marcas_del_tot
0,A4126AHA,72,39
1,A4126AHE,154,87
2,A4126AKR,164,81
3,A4190AAA,59,40
4,A4190ABG,189,85
...,...,...,...
15511,Z9405CJL,224,98
15512,Z9407CJF,122,64
15513,Z9407DAD,85,49
15514,Z9408ACL,98,57


## 2.5 Cuantos productos y Marcas del Mercado vende la farmacia

In [13]:
cant_mercado = data[data['Monto']>0].groupby(['cpa'],as_index = False)['pres_desc', 'Marca'].nunique()
cant_mercado.columns = ['cpa', 'desc_del_mkt', 'marcas_del_mkt']

C:\Users\rvozzi\AppData\Local\Temp/ipykernel_12160/1413015811.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cant_mercado = data[data['Monto']>0].groupby(['cpa'],as_index = False)['pres_desc', 'Marca'].nunique()


## 2.6 Facturacion de la Farmacia

### 2.6.1 Total

In [14]:
fact_tot = base.groupby(['cpa'], as_index = False)['Monto'].sum()
fact_tot.columns = ['cpa', 'fact_tot']

### 2.6.2 Mercado

In [15]:
pivot_fact = pd.pivot_table(base, 
                           values = 'Monto', 
                           index = 'cpa', 
                           columns = 'Mercado', 
                           aggfunc=np.sum, 
                           fill_value=0)
pivot_fact = pivot_fact.add_prefix('fact_mercado_')

In [16]:
# Junto estas 2 tablas
fact_tot = pd.merge(fact_tot,
                    pivot_fact,
                    left_on = 'cpa',
                    right_on = 'cpa',
                    how = 'inner')

In [17]:
#aca voy juntando todo

In [18]:
#asi resuelvo el 2.2

#aca agrego las shares a nivel pais
a = ['pres_desc', 'Marca', 'Segmento']
for hier in a:
    data = pd.merge(data,
                    shares_pais_hier(hier),
                    left_on = hier,
                    right_on = hier,
                    how = 'left')

#aca agrego las shares a nivel localidad y ruta
b = ['LOCALIDAD', 'RUTA']
for hier in a:
    for geo in b:
        data = pd.merge(data,
                        calculate_shares_geo_hier(hier, geo),
                        left_on = [hier, geo],
                        right_on = [hier, geo],
                        how = 'left')

In [19]:
# aca resuelvo el 2.3
data = pd.merge(data,
                    pivot_gen,
                    left_on = 'cpa',
                    right_on = 'cpa',
                    how = 'left')
# TODO: fillna con cero (son las farmacias donde no se venden ningun producto genomma)
data.fillna(value = 0, inplace = True)

In [20]:
# aca resuelvo el 2.4
data = pd.merge(data,
                    cant_total,
                    left_on = 'cpa',
                    right_on = 'cpa',
                    how = 'left')

In [21]:
# aca resuelvo el 2.5
data = pd.merge(data,
                    cant_mercado,
                    left_on = 'cpa',
                    right_on = 'cpa',
                    how = 'left')

In [22]:
# aca resuelvo el 2.6
data = pd.merge(data,
                    fact_tot,
                    left_on = 'cpa',
                    right_on = 'cpa',
                    how = 'inner')

In [23]:
#asi resuelvo el 2.1
data = pd.concat([data, onehot_vars], axis = 1)

In [24]:
#guardo el archivo del mercado
data.to_csv(path_or_buf = direc + mercado + '_' + 'data.csv',
                    sep = ',', index = False)

In [25]:
data

,cpa,Marca,Submarca,pres_desc,lab,Segmento,Monto,PROVINCIA,LOCALIDAD,RUTA,...,es_ruta__LITORAL,es_ruta__MENDOZA,es_ruta__NEA,es_ruta__NOA NORTE,es_ruta__NOA SUR,es_ruta__PATAGONIA,es_ruta__ROSARIO,es_visita__NO ATENDIDAS,es_visita__NO VISITADA,es_visita__VISITADA
0,A4126AHA,ACTRON,ACTRON 600 RAPIDA ACCION,ACTRON 600 RAPIDA ACCION 10 caps.blandas.,Bayer Consumer,PLUS,1692,SALTA,EL NARANJO,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
1,A4126AHA,ACTRON,ACTRON 600 RAPIDA ACCION,ACTRON 600 RAPIDA ACCION 20 caps.blandas.,Bayer Consumer,PLUS,4095,SALTA,EL NARANJO,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
2,A4126AHA,ACTRON,ACTRON PLUS,ACTRON PLUS caps.x 8,Bayer Consumer,MIGRA,276,SALTA,EL NARANJO,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
3,A4126AHA,ACTRON,ACTRON RAPIDA ACCION,ACTRON RAPIDA ACCION 10 caps.blandas.x 400mg.,Bayer Consumer,PLUS,236,SALTA,EL NARANJO,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
4,A4126AHA,ACTRON,ACTRON RAPIDA ACCION,ACTRON RAPIDA ACCION 20 caps.blandas.x 400mg.,Bayer Consumer,PLUS,450,SALTA,EL NARANJO,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889287,Z9408ACO,TAFIROL,TAFIROL 1G,TAFIROL 1G comp.ran.x 24,Genomma Lab.,1 GR,0,SANTA CRUZ,VEINTIOCHO DE NOVIEMBRE,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
889288,Z9408ACO,TAFIROL,TAFIROL FORTE,TAFIROL FORTE 650 mg comp.x 30,Genomma Lab.,FORTE,0,SANTA CRUZ,VEINTIOCHO DE NOVIEMBRE,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
889289,Z9408ACO,TAFIROL,TAFIROL,TAFIROL comp.x 100,Genomma Lab.,500 MG,0,SANTA CRUZ,VEINTIOCHO DE NOVIEMBRE,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
889290,Z9408ACO,TAFIROL,TAFIROL MIGRA,TAFIROL MIGRA comp.x 5,Genomma Lab.,MIGRA,0,SANTA CRUZ,VEINTIOCHO DE NOVIEMBRE,FUERA RUTA,...,0,0,0,0,0,0,0,1,0,0
